In [1]:
# setup dataset
from datasets import Dataset
import numpy as np
import faiss
retrieval_vector_size = 10

dataset = Dataset.from_dict(
    {
        "id": [str(i) for i in range(10)],
        "text": ["My favourite number is 3455", "The secret word is FROG"] * 5,
        "embeddings": [
            0.1 * np.ones(retrieval_vector_size),
            0.9 * np.ones(retrieval_vector_size),
        ] * 5,
    }
)

# We'd normally use faiss.METRIC_INNER_PRODUCT, but we're using METRIC_L2 here to make the results easier to understand
dataset.add_faiss_index("embeddings", metric_type=faiss.METRIC_L2)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'text', 'embeddings'],
    num_rows: 10
})

In [2]:
from src.transformers.models.atlas.retrieval_atlas import AtlasRetrieverIndex, AtlasConfig, AtlasTokenizer

config = AtlasConfig.from_pretrained("./data/atlas-pretrained")
tokenizer = AtlasTokenizer.from_pretrained("./data/atlas-pretrained", config=config)

retriever_index = AtlasRetrieverIndex(config, tokenizer, dataset)

In [3]:
# test retriever
retriever_hidden_states = 0.2 * np.zeros((1, retrieval_vector_size))
generator_input_ids = tokenizer.generator("hello world", return_tensors="pt")["input_ids"]

retrieved, _ = retriever_index(retriever_hidden_states, generator_input_ids, 5)

retrieved_strings = tokenizer.generator.batch_decode(retrieved["input_ids"][0])
print(retrieved_strings)
for string in retrieved_strings:
    assert "3455" in string

[[0.10000001 0.10000001 0.10000001 0.10000001 0.10000001]]
[{'embeddings': array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]], dtype=float32), 'id': ['0', '2', '4', '6', '8'], 'text': ['My favourite number is 3455', 'My favourite number is 3455', 'My favourite number is 3455', 'My favourite number is 3455', 'My favourite number is 3455'], 'score': array([0.10000001, 0.10000001, 0.10000001, 0.10000001, 0.10000001],
      dtype=float32)}]
['hello world context: My favourite number is 3455</s>', 'hello world context: My favourite number is 3455</s>', 'hello world context: My favourite number is 3455</s>', 'hello world context: My favourite number is 3455</s>', 'hello world context: My favourite number is 3455</s>']


In [4]:
# test hiding passages
# retriever_hidden_states = 0.2 * np.zeros((1, retrieval_vector_size))
# generator_input_ids = tokenizer.generator("hello world", return_tensors="pt")["input_ids"]
# retrieved, _ = retriever_index(retriever_hidden_states, generator_input_ids, 6, [str(i) for i in range(0,10,2)])
# retrieved_strings = tokenizer.generator.batch_decode(retrieved["input_ids"][0])
# print(retrieved_strings)
# for string in retrieved_strings:
#     assert "3455" not in string

[[0.10000001 0.10000001 0.10000001 0.10000001 0.10000001 8.099999  ]]
[{'embeddings': array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9]], dtype=float32), 'id': ['0', '2', '4', '6', '8', '1'], 'text': ['My favourite number is 3455', 'My favourite number is 3455', 'My favourite number is 3455', 'My favourite number is 3455', 'My favourite number is 3455', 'The secret word is FROG'], 'score': array([0.10000001, 0.10000001, 0.10000001, 0.10000001, 0.10000001,
       8.099999  ], dtype=float32)}]
['hello world context: My favourite number is 3455</s><pad>', 'hello world context: My favourite number is 3455</s><pad>', 'hello world context: My favourite number is 3455</s><pad>', 'hello world con

AssertionError: 

In [5]:
# test reindex
from transformers import AtlasModel
atlas = AtlasModel.from_pretrained('data/atlas-pretrained', retriever_index=retriever_index)

retriever_index.reindex(atlas, batch_size=2)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]